### Q1

In [42]:
p_red = 2 * 13/52

print("P(Red) =", p_red)

p_hearts_and_red = 13/52
p_hearts_given_red = p_hearts_and_red / p_red

print("P(Hearts | Red) =", p_hearts_given_red)

p_face_cards = 4 * 3/52
p_diamonds = 3/52
p_diamonds_given_face_cards = p_diamonds / p_face_cards

print("P(Diamond | Face) =", p_diamonds_given_face_cards)

p_spade_or_queen = (3 + 3) / 52
p_spade_or_queen_give_face_cards = p_spade_or_queen / p_face_cards
print("P(Spade OR Queen | Face) =", p_spade_or_queen_give_face_cards)



P(Red) = 0.5
P(Hearts | Red) = 0.5
P(Diamond | Face) = 0.25
P(Spade OR Queen | Face) = 0.5


### Q2

In [43]:
from pgmpy.models import DiscreteBayesianNetwork
from pgmpy.factors.discrete import TabularCPD
from pgmpy.inference import VariableElimination

In [44]:
model = DiscreteBayesianNetwork([
    ('intelligence', 'grade'),
    ('study hours','grade'),
    ('difficulty', 'grade'),
    ('grade', 'pass')
])


In [45]:
cpd_intelligence = TabularCPD(variable='intelligence',variable_card=2, values = [[0.7], [0.3]], state_names = {'intelligence': ['High', 'Low']})
cpd_study_hrs = TabularCPD(variable= "study hours", variable_card=2, values = [[0.6], [0.4]], state_names = {'study hours': ['Sufficient', 'Insufficient']})
cpd_difficulty = TabularCPD(variable = "difficulty", variable_card=2, values = [[0.4], [0.6]], state_names={'difficulty': ['Hard', 'Easy']})

#Grade (G) Node (3 Parents → 2³ = 8 Combinations)
#Parents: Intelligence (I), StudyHours (S), Difficulty (D)
#States: A, B, C

#I	      StudyHrs	    Diff	P(G=A)	P(G=B)	P(G=C)
#High	Sufficient	    Hard	0.5	    0.3 	0.2
#High	Sufficient	    Easy	0.8	    0.15	0.05
#High	Insufficient	Hard	0.3	    0.4	    0.3
#High	Insufficient	Easy	0.6	    0.3	    0.1
#Low	Sufficient	    Hard	0.2	    0.4	    0.4
#Low	Sufficient	    Easy	0.4	    0.4	    0.2
#Low	Insufficient	Hard	0.1	    0.3	    0.6
#Low	Insufficient	Easy	0.2	    0.4	    0.4

cpd_grade = TabularCPD( 
    variable='grade',
    variable_card=3, #A, B, C
    values=[
        [0.5, 0.8, 0.3, 0.6, 0.2, 0.4, 0.1, 0.2],  #A
        [0.3, 0.15, 0.4, 0.3, 0.4, 0.4, 0.3, 0.4], #B
        [0.2, 0.05, 0.3, 0.1, 0.4, 0.2, 0.6, 0.4]  #C
    ],
    evidence=['intelligence', 'study hours', 'difficulty'],
    evidence_card=[2,2,2],
    state_names={ #when defining state_names, child states first then parent names per evidence order
        'grade':['A', 'B', 'C'], #grade States: ['A', 'B', 'C'] → 0='A', 1='B', 2='C'.
            #Parent States:
            #intelligence: 0='High', 1='Low'
            #study_hours: 0='Sufficient', 1='Insufficient'
            #difficulty: 0='Hard', 1='Easy'
        'intelligence': ['High', 'Low'],
        'study hours': ['Sufficient', 'Insufficient'],
        'difficulty':['Hard', 'Easy']
    }
)
cpd_pass = TabularCPD( 
    variable="pass", 
    variable_card=2,
    values=[
        [0.95, 0.80, 0.50],  # P(Pass=Yes | G=A/B/C)
        [0.05, 0.20, 0.50]   # P(Pass=No | G=A/B/C)
    ], 
    evidence=['grade'], 
    evidence_card=[3],
    state_names={
        'pass': ['Yes', 'No'],
        'grade': ['A', 'B', 'C']
    }
)



In [46]:
model.add_cpds(cpd_difficulty, cpd_grade, cpd_intelligence, cpd_pass, cpd_study_hrs)

In [47]:
assert model.check_model()

In [48]:
inference = VariableElimination(model)

In [49]:
result = inference.query(variables = ['pass'], evidence={'study hours':'Sufficient', 'difficulty':'Hard'})

In [50]:
print(result)

+-----------+-------------+
| pass      |   phi(pass) |
+===========+=============+
| pass(Yes) |      0.7835 |
+-----------+-------------+
| pass(No)  |      0.2165 |
+-----------+-------------+


In [51]:
result = inference.query(variables=['intelligence'], evidence = {'pass':'Yes'})

In [52]:
print(result)

+--------------------+---------------------+
| intelligence       |   phi(intelligence) |
+====================+=====================+
| intelligence(High) |              0.7305 |
+--------------------+---------------------+
| intelligence(Low)  |              0.2695 |
+--------------------+---------------------+


### Q3

In [53]:
model = DiscreteBayesianNetwork([
    ('disease', 'fever'),
    ('disease', 'cough'),
    ('disease', 'fatigue'),
    ('disease', 'chills')
])

In [54]:
cpd_disease = TabularCPD(
    variable = "disease",
    variable_card = 2,
    values = [[0.3], [0.7]],
    state_names = {'disease':['Flu', 'Cold']}
)

cpd_fever = TabularCPD(
    variable = "fever",
    variable_card = 2,
    values = [[0.9, 0.5], 
              [0.1, 0.5]],
    evidence = ['disease'],
    evidence_card = [2],
    state_names = {
        'fever': ['Yes','No'],
        'disease': ['Flu', 'Cold']
    }
)
cpd_cough = TabularCPD(
    variable= 'cough',
    variable_card = 2,
    values = [[0.8, 0.6], # P(Cough=Yes | Disease=Flu, Disease=Cold)
              [0.2, 0.4]],# P(Cough=No | Disease=Flu, Disease=Cold)
    evidence = ['disease'],
    evidence_card = [2],
    state_names = {
        'cough': ['Yes','No'],
        'disease': ['Flu', 'Cold']
    }
)
cpd_fatigue = TabularCPD(
    variable= 'fatigue',
    variable_card = 2,
    values = [[0.7, 0.3], 
              [0.3, 0.7]],
    evidence = ['disease'],
    evidence_card = [2],
    state_names = {
        'fatigue': ['Yes','No'],
        'disease': ['Flu', 'Cold']
    }
)

cpd_chills = TabularCPD(variable= 'chills',
    variable_card = 2,
    values = [[0.6, 0.4], 
              [0.4, 0.6]],
    evidence = ['disease'],
    evidence_card = [2],
    state_names = {
        'chills': ['Yes','No'],
        'disease': ['Flu', 'Cold']
    }
)


In [56]:
model.add_cpds(cpd_chills, cpd_fatigue, cpd_cough, cpd_fever, cpd_disease)

In [68]:
assert model.check_model(), 'Model is invalid'

In [59]:
inference = VariableElimination(model)

In [61]:
result = inference.query(variables = ['disease'], evidence = {'fever':'Yes', 'cough':'Yes'})

In [62]:
print(result)

+---------------+----------------+
| disease       |   phi(disease) |
+===============+================+
| disease(Flu)  |         0.5070 |
+---------------+----------------+
| disease(Cold) |         0.4930 |
+---------------+----------------+


In [63]:
result = inference.query(variables = ['disease'], evidence = {'fever':'Yes', 'cough':'Yes', 'chills':'Yes'})

In [64]:
print(result)

+---------------+----------------+
| disease       |   phi(disease) |
+===============+================+
| disease(Flu)  |         0.6067 |
+---------------+----------------+
| disease(Cold) |         0.3933 |
+---------------+----------------+


In [66]:
result = inference.query(variables = ['fatigue'], evidence = {'disease':'Flu'})

In [67]:
print(result)

+--------------+----------------+
| fatigue      |   phi(fatigue) |
+==============+================+
| fatigue(Yes) |         0.7000 |
+--------------+----------------+
| fatigue(No)  |         0.3000 |
+--------------+----------------+
